# S18 T01 Base de dades NoSQL

In [1]:
import pymongo
import datetime
import pprint
from bson.objectid import ObjectId

import pandas as pd
import numpy as np

## Exercici 1
Crea una base de dades NoSQL utilitzant MongoDB. Afegeix-li algunes dades d'exemple que et permetin comprovar que ets capaç de processar-ne la informació de manera bàsica.

## Exercici 2
Connecta la base de dades NoSQL a Python utilitzant per exemple pymongo.

Both exercises are together because I created the database using pymongo.

In [2]:
# Connect to MongoDB and create the database 
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client["colegioXX"]

In [3]:
# Create a collection (like a table in relational DB)
estudiantes_col = db["estudiantes"]
print(estudiantes_col)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'colegioXX'), 'estudiantes')


In [5]:
# Insert one document in a collection
est1 = {"studID": 1,
         "name": "Juan Pérez",  
         "grades": {"math": 9, "history": 8, "science": 6}, 
         "date": datetime.datetime.utcnow()}

est1_id = estudiantes_col.insert_one(est1).inserted_id
est1_id

ObjectId('618be28b8fe6e2d3a9e4f19c')

In [6]:
# Make another insert
est2 = {"studID": 2,
         "name": "María Gómez",
         "grades": {"english": 6, "history": 9},
         "date": datetime.datetime.utcnow()}

est2_id = estudiantes_col.insert_one(est2).inserted_id
est2_id

ObjectId('618be28c8fe6e2d3a9e4f19d')

In [7]:
# Insert another element
est3 = {"studID": 3,
         "name": "Pablo Fernández",
         "grades": {"english": 6, "history": 9},
         "phone": "622791144",
         "date": datetime.datetime.utcnow()}

est3_id = estudiantes_col.insert_one(est3).inserted_id
est3_id

ObjectId('618be28d8fe6e2d3a9e4f19e')

In [8]:
# Create a new collection and add the data
profesores_col = db["profesores"]

prof1 = {"name": "Estela Fernández",
         "subjects": ["math", "geometry"],
         "age": 42,
         "phone": ["622794411", "633374466"]}

prof1_id = profesores_col.insert_one(prof1).inserted_id
prof1_id

ObjectId('618be28e8fe6e2d3a9e4f19f')

In [9]:
more_profs = [{"name": "Ana María García",
               "subjects": ["history", "literature", "english"],
               "age": 32,
               "gender": "female"},
              {"name": "Marta Pons",
               "subjects": ["phy-ed", "math"],
               "age": 28,
               "phone": "623455877",
               "gender": "female",
               "degree": "Phd"},
              {"name": "Martín Grassi",
               "subjects": ["phy-ed", "math"],
               "age": 44,
               "phone": "623448877",
               "gender": "male"}]

result = profesores_col.insert_many(more_profs)
result.inserted_ids

[ObjectId('618be28e8fe6e2d3a9e4f1a0'),
 ObjectId('618be28e8fe6e2d3a9e4f1a1'),
 ObjectId('618be28e8fe6e2d3a9e4f1a2')]

In [10]:
# What collections are in the database?
db.list_collection_names()

['profesores', 'estudiantes']

In [11]:
# Search for a name in the profesores collection
pprint.pprint(profesores_col.find_one({"phone": "623448877"})) # find the first ocurrence

{'_id': ObjectId('618be28e8fe6e2d3a9e4f1a2'),
 'age': 44,
 'gender': 'male',
 'name': 'Martín Grassi',
 'phone': '623448877',
 'subjects': ['phy-ed', 'math']}


In [12]:
# Search for profesores with a particular subject
for prof in profesores_col.find({"subjects": "math"}):
    pprint.pprint(prof)

{'_id': ObjectId('618be28e8fe6e2d3a9e4f19f'),
 'age': 42,
 'name': 'Estela Fernández',
 'phone': ['622794411', '633374466'],
 'subjects': ['math', 'geometry']}
{'_id': ObjectId('618be28e8fe6e2d3a9e4f1a1'),
 'age': 28,
 'degree': 'Phd',
 'gender': 'female',
 'name': 'Marta Pons',
 'phone': '623455877',
 'subjects': ['phy-ed', 'math']}
{'_id': ObjectId('618be28e8fe6e2d3a9e4f1a2'),
 'age': 44,
 'gender': 'male',
 'name': 'Martín Grassi',
 'phone': '623448877',
 'subjects': ['phy-ed', 'math']}


In [13]:
# Find a document using its id
data = estudiantes_col.find_one({'_id': ObjectId(est3_id)})
data

{'_id': ObjectId('618be28d8fe6e2d3a9e4f19e'),
 'studID': 3,
 'name': 'Pablo Fernández',
 'grades': {'english': 6, 'history': 9},
 'phone': '622791144',
 'date': datetime.datetime(2021, 11, 10, 15, 17, 33, 180000)}

In [21]:
# Find professors name starting with M
myquery = { "name": { "$gt": "M" } }

mydoc = profesores_col.find(myquery)
for prof in mydoc:
    print(prof)

{'_id': ObjectId('618be28e8fe6e2d3a9e4f1a1'), 'name': 'Marta Pons', 'subjects': ['phy-ed', 'math'], 'age': 28, 'phone': '623455877', 'gender': 'female', 'degree': 'Phd'}
{'_id': ObjectId('618be28e8fe6e2d3a9e4f1a2'), 'name': 'Martín Grassi', 'subjects': ['phy-ed', 'math'], 'age': 44, 'phone': '623448877', 'gender': 'male'}


In [28]:
# List the students and their telephone numbers
for x in estudiantes_col.find({},{ "_id": 0, "name": 1, "phone": 1 }):
  print(x)

{'name': 'Juan Pérez'}
{'name': 'María Gómez'}
{'name': 'Pablo Fernández', 'phone': '622791144'}


## Exercici 3
Carega algunes consultes senzilles a un Pandas Dataframe. 

In [16]:
# Create a profesores dataframe
df_prof = pd.DataFrame(columns=('prof_code', 'var', 'value'))

# Define a unique code por each professor
code = 1

# Define a function to add values of variables in the dataframe
def agregar(df, code, var, val):
    new = {'prof_code': code, 'var': var, 'value': val}
    df = df.append(new, ignore_index=True)
    return df
    

# Search for every document in the collection profesores
for prof in profesores_col.find():
    for p in prof:
        if type(prof[p]) is list:
            for val in prof[p]:
                df_prof = agregar(df_prof, code, p, val)
        else:
            df_prof = agregar(df_prof, code, p, prof[p])
            
    code += 1
df_prof

,prof_code,var,value
0,1,_id,618be28e8fe6e2d3a9e4f19f
1,1,name,Estela Fernández
2,1,subjects,math
3,1,subjects,geometry
4,1,age,42
5,1,phone,622794411
6,1,phone,633374466
7,2,_id,618be28e8fe6e2d3a9e4f1a0
8,2,name,Ana María García
9,2,subjects,history


## Exercici 4
Genera un resum estadístic de la informació que conté la base de dades.

In [24]:
# Which collections are in the database?
collist = db.list_collection_names()
for col in collist:
    print("Collection name:",col)
    

Collection name: profesores
Collection name: estudiantes


In [14]:
# How many documents has the collection?
print(profesores_col.count_documents({}))

4


In [15]:
print(estudiantes_col.count_documents({}))

3


In [26]:
# What is the average age of the professors?
suma = 0
count = 0
for prof in profesores_col.find():
    for p in prof:
        if p == "age":
            suma = suma + prof[p]
            count += 1
        
print("Average age is :", round(suma/count,1)) 

Average is : 36.5


In [43]:
# The average coincides with the one in the dataframe
df_prof['value'][df_prof['var'] =='age'].mean()

36.5